<a href="https://colab.research.google.com/github/zhangluustb/transformers_notebook/blob/main/trocr_chinese_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chineseocr/trocr-chinese.git

fatal: destination path 'trocr-chinese' already exists and is not an empty directory.


In [2]:
import os
os.chdir("/content/trocr-chinese")

In [3]:
pwd

'/content/trocr-chinese'

In [5]:
# install
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 58.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.6 MB/s eta 0:00:00
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_

In [6]:
# copy weights
!cp /content/drive/MyDrive/trocr-chinese/pre-weights/TAL_OCR_CHN.zip .

In [7]:
!unzip TAL_OCR_CHN.zip

Archive:  TAL_OCR_CHN.zip
  inflating: TAL_OCR_CHN/config.json  
 extracting: TAL_OCR_CHN/merges.txt  
  inflating: TAL_OCR_CHN/preprocessor_config.json  
  inflating: TAL_OCR_CHN/pytorch_model.bin  
  inflating: TAL_OCR_CHN/special_tokens_map.json  
  inflating: TAL_OCR_CHN/tokenizer_config.json  
  inflating: TAL_OCR_CHN/vocab.json  
  inflating: TAL_OCR_CHN/vocab.txt   


In [ ]:
# 测试训练脚本
!python init_custdata_model.py --cust_vocab ./cust-data/vocab.txt --pretrain_model ./weights --cust_data_init_weights_path ./cust-data/weights

In [ ]:
!python train.py --cust_data_init_weights_path ./cust-data/weights --checkpoint_path ./checkpoint/trocr-custdata --dataset_path "./dataset/cust-data/*/*.jpg" --per_device_train_batch_size 1 --CUDA_VISIBLE_DEVICES 0

---

# 分析代码

## 模型和预训练模型

In [24]:
import os
import json
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers import AutoConfig
def read_vocab(vocab_path):
    """
    读取自定义训练字符集
    vocab_path format:
    1\n
    2\n
    ...
    我\n
    """
    other = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
    vocab = {}
    for ot in other:
        vocab[ot] = len(vocab)

    with open(vocab_path) as f:
        for line in f:
            line = line.strip('\n')
            if line not in vocab:
                vocab[line] = len(vocab)
    return vocab
pretrain_model="weights"
cust_data_init_weights_path="new_weights"


processor = TrOCRProcessor.from_pretrained(pretrain_model)
pre_model = VisionEncoderDecoderModel.from_pretrained(pretrain_model)

pre_vocab = processor.tokenizer.get_vocab()
cust_vocab = read_vocab("cust-data/vocab.txt")

keep_tokens = []
unk_index = pre_vocab.get('<unk>')
for key in cust_vocab:
    keep_tokens.append(pre_vocab.get(key, unk_index))

processor.save_pretrained("new_weights")

pre_model.save_pretrained("new_weights")
## 替换词库
with open(os.path.join("new_weights", "vocab.json"), "w") as f:
    f.write(json.dumps(cust_vocab, ensure_ascii=False))

##替换模型参数
with open(os.path.join("new_weights", "config.json")) as f:
    model_config = json.load(f)


In [25]:
len(keep_tokens)

3081

In [27]:
## 替换roberta embedding层词库
model_config["decoder"]['vocab_size'] = len(cust_vocab)

## 替换 attetion 字库
model_config['vocab_size'] = len(cust_vocab)

with open(os.path.join(cust_data_init_weights_path, "config.json"), "w") as f:
    f.write(json.dumps(model_config, ensure_ascii=False))

##加载cust model
cust_config = AutoConfig.from_pretrained(cust_data_init_weights_path)
cust_model = VisionEncoderDecoderModel(cust_config)

pre_model_weigths = pre_model.state_dict()
cust_model_weigths = cust_model.state_dict()

In [28]:
len(keep_tokens)

3081

In [29]:
##权重初始化
print("loading init weights..................")
for key in pre_model_weigths:
    
    if pre_model_weigths[key].shape != cust_model_weigths[key].shape:
        print("name:", key,pre_model_weigths[key].shape)
        wt = pre_model_weigths[key][keep_tokens, :] #未知的token设置为unk权重
        cust_model_weigths[key] = wt
    else:
        cust_model_weigths[key] = pre_model_weigths[key]

loading init weights..................
name: decoder.model.decoder.embed_tokens.weight torch.Size([2667, 256])
name: decoder.output_projection.weight torch.Size([2667, 256])


In [31]:
len(set(keep_tokens))

2001

In [32]:
cust_model_weigths["decoder.model.decoder.embed_tokens.weight"].shape

torch.Size([3081, 256])

In [34]:
pre_vocab.get('<unk>')

3

---

## model and train

trocrDataset

```
# python 
image = Image.open(image_file).convert("RGB")
image = self.transformer(image) ##图像增强函数
pixel_values = self.processor(image, return_tensors="pt").pixel_values
labels = encode_text(text, max_target_length=self.max_target_length, vocab=self.vocab)
labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
```

encode
```
# python
text = text[:max_target_length - 2]
tokens = [vocab.get('<s>')]
unk = vocab.get('<unk>')
pad = vocab.get('<pad>')
mask = []
for tk in text:
    token = vocab.get(tk, unk)
    tokens.append(token)
    mask.append(1)

tokens.append(vocab.get('</s>'))
mask.append(1)

if len(tokens) < max_target_length:
    for i in range(max_target_length - len(tokens)):
        tokens.append(pad)
        mask.append(0)
```



In [40]:
cust_data_init_weights_path="./new_weights"
checkpoint_path ="./checkpoint/trocr-custdata"
dataset_path="./dataset/cust-data/*/*.jpg"
per_device_train_batch_size=1

In [41]:
processor = TrOCRProcessor.from_pretrained(cust_data_init_weights_path)
vocab = processor.tokenizer.get_vocab()
vocab_inp = {vocab[key]: key for key in vocab}

In [43]:
from glob import glob
train_paths=glob(dataset_path)
test_paths=train_paths

In [46]:
from dataset import trocrDataset, decode_text
transformer = lambda x: x
train_dataset = trocrDataset(paths=train_paths, processor=processor, max_target_length=128, transformer=transformer)
transformer = lambda x: x  ##图像数据增强函数
eval_dataset = trocrDataset(paths=test_paths, processor=processor, max_target_length=128, transformer=transformer)


In [48]:
model = VisionEncoderDecoderModel.from_pretrained(cust_data_init_weights_path)
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size

model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 256
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

RuntimeError: ignored



```
training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=args.per_device_train_batch_size,
        per_device_eval_batch_size=8,
        fp16=True,
        output_dir=args.checkpoint_path,
        logging_steps=10,
        num_train_epochs=args.num_train_epochs,
        save_steps=args.eval_steps,
        eval_steps=args.eval_steps,
        save_total_limit=5
    )

# seq2seq trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()
trainer.save_model(os.path.join(args.checkpoint_path, 'last'))
processor.save_pretrained(os.path.join(args.checkpoint_path, 'last'))

```



---

## 模型结构分析

参考transformers github